In [ ]:
from IPython.display import Markdown as md
import pegasus as pg
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
data = pg.read_input(snakemake.input[0])

In [ ]:
for resolution in snakemake.config["preprocess"]["resolutions"]:
    print(f".........Clustering for resolution: {resolution}.........")
    pg.leiden(
        data=data,
        rep="scanorama_regressed",
        resolution=resolution,
        class_label=f"leiden_label_{resolution}",
    )
    pg.scatter(data, attrs=[f"leiden_label_{resolution}", "Channel"], basis="umap_pca")
    pg.dendrogram(data=data, groupby=f"leiden_label_{resolution}")
    pg.compo_plot(
        data=data,
        groupby=f"leiden_label_{resolution}",
        condition="Channel",
        n_jobs=snakemake.threads,
    )
    pg.de_analysis(data=data, cluster=f"leiden_label_{resolution}")
    celltype_dict = pg.infer_cell_types(data=data, markers="human_brain")
    cluster_names = pg.infer_cluster_names(celltype_dict)
    pg.annotate(
        data=data,
        name=f"leiden_label_{resolution}_cell_types",
        based_on=f"leiden_label_{resolution}",
        anno_dict=cluster_names,
    )

In [ ]:
pg.write_output(data, snakemake.output["h5ad"])